In [145]:
#importing required libraries
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver

import warnings
warnings.filterwarnings('ignore')

# Scrapping data from amazon.com

In [146]:
#creating a function for searching page no's in browser
def page_temp(search_bar,page_no=1):
    """Generate a URL for search Term with page number"""
    search_bar = search_bar.replace(' ','+')
    temp = f'https://www.amazon.in/s?k={search_bar}&page={page_no}&qid=1623864210&ref=sr_pg_{page_no}'
    
    return temp

In [147]:
#creating a function that will generate URLs with the search
def fetch_urls():
    scrap_items = ['laptops', 'Phones', 'Headphones', 'smart watches', 'Professional Cameras', 'Printers', 'scanners', 'keyboard', 'monitors', 'Home theater', 'router']
    URLs = []     
    for i in scrap_items:
        for j in range(1,11):
            URLs.append(page_temp(i,j))
    return URLs        

In [148]:
#Creating a function that scraps the URLs of the front page of the reviews and rattings with the search
def review_urls():
    
    url_links = []#store scrap data into this variable.
    driver = webdriver.Chrome(r'../Web scrapping/chromedriver 2')
    driver.maximize_window()

    URL = fetch_urls()
    for i in URL:
        driver.get(i)

        for i in driver.find_elements_by_xpath("//*[@class='a-link-normal s-link-style a-text-normal']"):
            url_links.append(i.get_attribute("href"))
    driver.close()    
    return url_links

In [149]:
url_links = review_urls()

In [150]:
url_links

['https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A050239413215IGNLPTBS&url=%2FAcer-AN515-57-i5-11400H-Graphics-Keyboard%2Fdp%2FB09B2F2DMK%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dlaptops%26qid%3D1641470373%26sr%3D8-1-spons%26psc%3D1&qualifier=1641470373&id=2710981152303781&widgetName=sp_atf',
 'https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A0920238GM3HWTBWOPYY&url=%2FHP-inches-Laptop-Windows-15s-gy0501AU%2Fdp%2FB09FL5CK19%2Fref%3Dsr_1_2_sspa%3Fkeywords%3Dlaptops%26qid%3D1641470373%26sr%3D8-2-spons%26psc%3D1&qualifier=1641470373&id=2710981152303781&widgetName=sp_atf',
 'https://www.amazon.in/Lenovo-IdeaPad-Celeron-Platinum-81VT0071IN/dp/B09G9ZPV38/ref=sr_1_3?keywords=laptops&qid=1641470373&sr=8-3',
 'https://www.amazon.in/Lenovo-IdeaPad-Celeron-Platinum-81WH007KIN/dp/B09MM4FPMR/ref=sr_1_4?keywords=laptops&qid=1641470373&sr=8-4',
 'https://www.amazon.in/Lenovo-IdeaPad-Warranty-Platinum-81WB01BCIN

In [151]:
len(url_links)

2267

In [152]:
#empty variables
Rattings = []
Reviews = []

In [153]:
#Creating a function that scraps the reviews and rattings from url_links.
def rev_rat():
    
    driver = webdriver.Chrome(r'../Web scrapping//chromedriver 2')
    driver.maximize_window()


    for i in url_links:
        driver.get(i)
        
        try:
            L1 = driver.find_element_by_xpath("//a[@data-hook='see-all-reviews-link-foot']")
            L1 = L1.get_attribute("href")
            L1 = L1.split('ref=cm')[0]
        except:
            pass     
        
        for i in range(1,5):
            L2= f'ref=cm_cr_getr_d_paging_btm_next_{i}?ie=UTF8&reviewerType=all_reviews&pageNumber={i}'
            L = L1+ L2 
            
            
            driver.get(L)
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            results = soup.find_all('div',{'class' : 'a-section celwidget'})
        
            for item in results:
                    try:
                        rev_ = item.find('span', {'data-hook': "review-body"})
                        a_tag = item.find(['a'], class_ = "a-link-normal")
                        if rev_.span.text.replace('\n','').strip() and a_tag['title'][0]:
                            Reviews.append(rev_.span.text.replace('\n','').strip())
                            Rattings.append(a_tag['title'][0])
                        else:
                            break
                    except:
                        break
                        
       
    driver.close()
    
    data = pd.DataFrame({'Rattings':Rattings, "Reviews": Reviews})
    
    #Saving the dataframe in csv file.
    data.to_csv("Rattings_data.csv",index=False) #Creating CSV
    
    return data

In [154]:
data = rev_rat()

In [155]:
data

,Rattings,Reviews
0,5,- I am using this laptop for about 15 days and...
1,1,I got a faulty product the charger wasn’t work...
2,5,one of the best thermals out there intels new ...
3,5,Best gaming laptop in this range even after lo...
4,5,"Loved this Laptop, Simply Amazing, Go for it"
...,...,...
6265,3,1. It is not at all rubbery.2. You won't be ab...
6266,5,Fits well and soft . Prevents dirt from gettin...
6267,5,This has very good fit for Asus TUF A17 2021 m...
6268,5,Fits the 2021 A15 rtx 3060 eclipse grey varian...
